In [1]:
import datetime

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql.window import Window

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
7,application_1668702798705_0008,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
df = spark.read.parquet("s3://tvlp-ds-users/kendra-frederick/shopping-grid/engineered-features/parquet/2022114/") 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- market: string (nullable = true)
 |-- out_departure_date: long (nullable = true)
 |-- shop_date: long (nullable = true)
 |-- fare_rank: integer (nullable = true)
 |-- num_itin: long (nullable = true)
 |-- fare: double (nullable = true)
 |-- out_cxrs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- solution_counts: long (nullable = true)
 |-- shop_counts: long (nullable = true)
 |-- avg_out_avail_max: double (nullable = true)
 |-- avg_out_avail_low: double (nullable = true)
 |-- outDeptDt_dt: date (nullable = true)
 |-- shopDate_dt: date (nullable = true)
 |-- days_til_dept: integer (nullable = true)
 |-- dept_dt_dow_int: integer (nullable = true)
 |-- dept_dt_dow: string (nullable = true)
 |-- fr2_fare: double (nullable = true)
 |-- fr2_out_cxrs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- fr2_avg_out_avail_max: double (nullable = true)
 |-- fr2_avg_out_avail_low: double (nullable = true)
 |-- fr1_fr2_out_cxrs_same

In [7]:
# df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['market', 'out_departure_date', 'shop_date', 'fare_rank', 'num_itin', 'fare', 'out_cxrs', 'solution_counts', 'shop_counts', 'avg_out_avail_max', 'avg_out_avail_low', 'outDeptDt_dt', 'shopDate_dt', 'days_til_dept', 'dept_dt_dow_int', 'dept_dt_dow', 'fr2_fare', 'fr2_out_cxrs', 'fr2_avg_out_avail_max', 'fr2_avg_out_avail_low', 'fr1_fr2_out_cxrs_same', 'fr1_fr2_out_cxrs_overlap', 'avg_fare_dtd', 'fare_prev_shop_day', 'fr2_fare_prev_shop_day', 'avg_out_avail_max_prev_shop_day', 'avg_out_avail_low_prev_shop_day', 'num_itin_prev_shop_day', 'shop_counts_prev_shop_day', 'solution_counts_prev_shop_day', 'fare_prev_dept_day', 'fare_next_dept_day', 'fare_diff_prev_dept_day', 'fare_diff_next_dept_day', 'trailing_avg_shop_counts', 'trailing_avg_solution_counts', 'trailing_avg_fare', 'trailing_std_fare', 'avg_diff_prev_dept_day', 'avg_diff_next_dept_day', 'est_fare_from_prev_dept_day', 'est_fare_from_next_dept_day']

In [8]:
# df.select("market").distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14

# define features

In [3]:
target_col = 'fare'

# scale: mean = 0, std = 1
features_to_scale = [
    # fare-based features
    'fare_prev_shop_day',
    'fr2_fare_prev_shop_day', 
    'avg_fare_dtd',
    'fare_prev_dept_day', 'fare_next_dept_day',
    'est_fare_from_prev_dept_day', 'est_fare_from_next_dept_day',
    'trailing_avg_fare', 'trailing_std_fare',
                    ]
# norm: [0,1]
features_to_norm = [
    'days_til_dept', # encoded in 'avg_fare_dtd', so exclude?
    'num_itin_prev_shop_day',
    # count-based features - expect some skew in these, so norm instead of scale (?)
    'shop_counts_prev_shop_day', 
    'trailing_avg_shop_counts',
    'solution_counts_prev_shop_day',
    'trailing_avg_solution_counts',
]
categorial_features = [
    'dept_dt_dow_int' 
]
as_is_features = [
    # avail-based features are already norm'd
    'avg_out_avail_max_prev_shop_day',
    'avg_out_avail_low_prev_shop_day', 
]
all_features = features_to_scale + features_to_norm + categorial_features + as_is_features
all_cols_for_modeling = all_features + [target_col]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# prep data

## clip

based on shop date (given that we took a 3-day trailing average -- the first 2 days won't have valid data)

In [4]:
first_shop_date = datetime.date(2022,10,15)
window_size = 3
shop_cutoff = first_shop_date + datetime.timedelta(days=window_size)
df_clip_shop = df.filter(F.col('shopDate_dt') >= shop_cutoff)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Limit to a certain number out into the future -- such as 120

In [24]:
# dtd_limit = 120
# df_clip_dtd = df_clip_shop.filter(F.col("days_til_dept") <= 120)
# df_clip_dtd.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

45548

This cuts out a lot of data, actually

## dealing with nulls

In [5]:
for col in all_cols_for_modeling:
    print(col, df_clip_shop.filter(F.col(col).isNull()).count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

fare_prev_shop_day 415
fr2_fare_prev_shop_day 2509
avg_fare_dtd 48
fare_prev_dept_day 392
fare_next_dept_day 392
est_fare_from_prev_dept_day 392
est_fare_from_next_dept_day 392
trailing_avg_fare 415
trailing_std_fare 415
days_til_dept 0
num_itin_prev_shop_day 415
shop_counts_prev_shop_day 415
trailing_avg_shop_counts 415
solution_counts_prev_shop_day 415
trailing_avg_solution_counts 415
dept_dt_dow_int 0
avg_out_avail_max_prev_shop_day 415
avg_out_avail_low_prev_shop_day 415
fare 0

In [23]:
col = 'fare_prev_shop_day'
df_null = df_clip_shop.filter(F.col(col).isNull())
df_null.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+---------+---------+--------+-------+--------+---------------+-----------+------------------+------------------+------------+-----------+-------------+---------------+-----------+--------+------------+---------------------+---------------------+---------------------+------------------------+------------------+------------------+----------------------+-------------------------------+-------------------------------+----------------------+-------------------------+-----------------------------+------------------+------------------+-----------------------+-----------------------+------------------------+----------------------------+-----------------+-----------------+----------------------+----------------------+---------------------------+---------------------------+
| market|out_departure_date|shop_date|fare_rank|num_itin|   fare|out_cxrs|solution_counts|shop_counts| avg_out_avail_max| avg_out_avail_low|outDeptDt_dt|shopDate_dt|days_til_dept|dept_dt_dow_int|de

A lot of these have high values for days til dept. Which makes sense. 

Either drop nulls for now, or collapse fare-based features into one or a few, e.g. taking an average that ignores the nulls

In [14]:
df.count(), df_clip_shop.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(133001, 119216)

In [6]:
df_no_nulls = df_clip_shop.dropna(subset=all_cols_for_modeling)
df_no_nulls.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

115776

In [26]:
115776 / 119216

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.9711448127768085

Drop for now. Revisit later

## Train-test split

In [7]:
train, test = df_no_nulls.randomSplit([0.8, 0.2], seed=19)
# train.count(), test.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
train_shuffled = train.orderBy(F.rand())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# assemble & scale/encode features

## import things

In [12]:
from pyspark.ml.feature import (
    MinMaxScaler, StandardScaler, OneHotEncoderEstimator,
    VectorAssembler, VectorIndexer
)
from pyspark.ml.regression import (
    LinearRegression, DecisionTreeRegressor
)
from pyspark.ml.evaluation import RegressionEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
from pyspark.ml import Pipeline

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
# ## OneHotEncoding

# categorial_features = [
#     # note: this must be of type numeric, not string. Spark MLlib is lame.
#     'dept_dt_dow_int' 
# ]

# ohe = OneHotEncoderEstimator(
#     inputCols=categorial_features,
#     outputCols=['dept_dow_enc']
# )
# ohe_model = ohe.fit(train)
# df_enc = ohe_model.transform(train)

# df_enc.select("out_departure_date", "dept_dt_dow", "dept_dow_enc").show()

# ## Scale

# # scale_models = []
# # for col in features_to_scale:
# #     scaler = StandardScaler(inputCol=col, outputCol=f"{col}_scaled")
# #     scale_model = scaler.fit(df_clip_shop)
# #     df_enc = df_enc.transform(df_enc)
    

# scale_assembler = VectorAssembler(
#     inputCols=features_to_scale,
#     outputCol="features_to_scale"
# )
# df_enc2 = scale_assembler.transform(df_enc)

# scaler = StandardScaler(
#     inputCol=scale_assembler.getOutputCol(),
#     outputCol="scaled_features"
# )
# scale_model = scaler.fit(df_enc2)
# df_enc2 = scale_model.transform(df_enc2)

# df_enc2.select(*features_to_scale, "features_to_scale", "scaled_features").show(10, truncate=False)

# (df_enc2.orderBy(F.desc("trailing_std_fare"))
#  .select(*features_to_scale, "features_to_scale", "scaled_features")
#  .show(10, truncate=False)
# )

# features_to_scale

# i = 2
# vs = VectorSlicer(inputCol='scaled_features', outputCol='ex', indices=[i])
# ex_output = vs.transform(df_enc2)
# ex_output.select(features_to_scale[i], 'ex').show(truncate=False)

# (ex_output
#  .orderBy(F.desc(features_to_scale[i]))
#  .select(features_to_scale[i], 'ex')
#  .show(10, truncate=False)
# )

In [87]:
# ## resume
# MinMax Scale instead of norm

# norm_assembler = VectorAssembler(
#     inputCols=features_to_norm,
#     outputCol="features_to_norm"
# )
# df_enc3 = norm_assembler.transform(df_enc2)

# normer = MinMaxScaler(
#     inputCol=norm_assembler.getOutputCol(),
#     outputCol="normed_features"
# )
# norm_model = normer.fit(df_enc3)
# df_enc3 = norm_model.transform(df_enc3)

# vs = VectorSlicer(inputCol='normed_features', outputCol='ex', indices=[1])
# ex_output = vs.transform(df_enc3)
# ex_output.select("num_itin_prev_shop_day", 'ex').show(truncate=False)

# ex_output.orderBy(F.desc("num_itin_prev_shop_day")).select("num_itin_prev_shop_day", 'ex').show(10, truncate=False)

# ## final assembly

# df_enc3.columns

# final_assembler = VectorAssembler(
#     inputCols=['dept_dow_enc', 'scaled_features', 'normed_features'] + as_is_features,
#     outputCol='features'
# )
# df_enc_final = final_assembler.transform(df_enc3)

# df_enc_final.select('features').show(10, truncate=False)

# ohe.getOutputCols()

# new_list = [*ohe.getOutputCols(), 
#                scaler.getOutputCol(),
#                normer.getOutputCol()
#               ] + as_is_features

# new_list

# Pipeline

In [13]:
ohe = OneHotEncoderEstimator(
    inputCols=categorial_features,
    outputCols=['dept_dow_enc']
)

scale_assembler = VectorAssembler(
    inputCols=features_to_scale,
    outputCol="features_to_scale"
)

scaler = StandardScaler(
    inputCol=scale_assembler.getOutputCol(),
    outputCol="scaled_features"
)

norm_assembler = VectorAssembler(
    inputCols=features_to_norm,
    outputCol="features_to_norm"
)

normer = MinMaxScaler(
    inputCol=norm_assembler.getOutputCol(),
    outputCol="normed_features"
)

# Note: if shuffling data doesn't work, add a VectorSizeHint here
# https://spark.apache.org/docs/2.4.8/ml-features.html#vectorsizehint
# I think this is what's going on: https://www.matthewslipper.com/2017/10/01/watch-out-vectorassembler-incompatible-spark-streaming.html
final_assembler = VectorAssembler(
    inputCols=[*ohe.getOutputCols(), 
               scaler.getOutputCol(),
               normer.getOutputCol()
              ] + as_is_features,
    outputCol='features'
)

regression_model = LinearRegression(
    #  elasticNetParam corresponds to α (ratio) and regParam corresponds to λ (weight).
    labelCol="fare",
    maxIter=20,
    # will need to tune these
    regParam=0.1,
    elasticNetParam=0.5
)

pipeline = Pipeline(stages=[
    scale_assembler,
    norm_assembler,
    ohe, 
    scaler,
    normer,
    final_assembler,
    regression_model
])

model = pipeline.fit(train_shuffled)
test_tx = model.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## evalu

In [121]:
evaluator = RegressionEvaluator(
    labelCol="fare"
)
# guess this is MSE?
evaluator.evaluate(test_tx)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

147.86357971629198

In [123]:
# evaluator.evaluate(test_tx, {evaluator.metricName: "mape"})
evaluator.evaluate(test_tx, {evaluator.metricName: "r2"})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.651822747779349

In [124]:
evaluator.evaluate(test_tx, {evaluator.metricName: "mae"})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

35.36802506337732

In [130]:
# calculate MAPE by hand
test_tx = (test_tx
           .withColumn("pred_err", F.col("fare") - F.col("prediction"))
           .withColumn("abs_pred_err", F.abs(F.col("pred_err")))
           .withColumn("abs_pct_err", F.col("abs_pred_err") / F.col("fare"))
          )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [133]:
test_tx.select("market", "out_departure_date", "shop_date", "fare", "prediction", "pred_err", "abs_pred_err", "abs_pct_err").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+---------+------+------------------+-------------------+------------------+--------------------+
| market|out_departure_date|shop_date|  fare|        prediction|           pred_err|      abs_pred_err|         abs_pct_err|
+-------+------------------+---------+------+------------------+-------------------+------------------+--------------------+
|ATL-EWR|          20221025| 20221019| 72.59|58.912710815737555| 13.677289184262449|13.677289184262449| 0.18841836594933803|
|ATL-EWR|          20221026| 20221021|  68.6| 57.98717889417168| 10.612821105828317|10.612821105828317|  0.1547058470237364|
|ATL-EWR|          20221027| 20221019| 72.59| 67.79875862802177|  4.791241371978231| 4.791241371978231| 0.06600415170103638|
|ATL-EWR|          20221027| 20221020|  68.6| 66.75598195841206| 1.8440180415879297|1.8440180415879297|0.026880729469211804|
|ATL-EWR|          20221027| 20221021|  68.6|  64.0027741120784|  4.597225887921596| 4.597225887921596|  0.0670149546344256|


In [19]:
def calc_abs_pct_err(df, pred_col="prediction"):
    df = (df
           .withColumn("pred_err", F.col("fare") - F.col(pred_col))
           .withColumn("abs_pred_err", F.abs(F.col("pred_err")))
           .withColumn("abs_pct_err", F.col("abs_pred_err") / F.col("fare"))
          )
    return df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-19:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [134]:
test_tx.select(F.mean("abs_pct_err").alias("mape")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+
|               mape|
+-------------------+
|0.14932619204051534|
+-------------------+

In [137]:
test_tx.groupBy("market").agg(F.mean("abs_pct_err").alias("mape")).orderBy("mape").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+
| market|                mape|
+-------+--------------------+
|LHR-EWR|0.045151099809955415|
|LHR-JFK|0.060324624848442476|
|EWR-LHR| 0.08656433938057669|
|LHR-LAX| 0.10295271078710452|
|JFK-LHR|   0.113395665248697|
|LAX-EWR| 0.12455392098372574|
|LAX-JFK| 0.12836529342361255|
|JFK-LAX| 0.12958263317090377|
|EWR-CDG| 0.13332879085323493|
|ATL-EWR| 0.15042672584419317|
|LGA-MIA|  0.2291117225446786|
|LAX-SFO| 0.25686493430822466|
|SFO-LAX| 0.26226365351769315|
|OAK-LAS|   0.421095149942065|
+-------+--------------------+

# tune hyperparams

In [14]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
param_grid = (ParamGridBuilder()
              # regParam is lambda (weight / factor), elasticNet is alpha (ratio)
              # note alpha is the opposite of sklearn's defintion
              .addGrid(regression_model.elasticNetParam, [0.01, 0.1, 0.5, 0.9, 0.99])
              .addGrid(regression_model.regParam, [0.01, 0.1, 1, 10, 100])
              .build()
             )

cv = CrossValidator(estimator=pipeline,
                    estimatorParamMaps=param_grid,
                    evaluator=RegressionEvaluator(labelCol="fare"),
                    numFolds=3,
                    parallelism=5
                   )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
cv_model = cv.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-16:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [17]:
pred_cv = cv_model.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-17:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [18]:
pred_cv.select("market", "out_departure_date", "shop_date", "fare", "prediction").show() #, "pred_err", "abs_pred_err", "abs_pct_err").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-18:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------+------------------+---------+------+------------------+
| market|out_departure_date|shop_date|  fare|        prediction|
+-------+------------------+---------+------+------------------+
|ATL-EWR|          20221025| 20221019| 72.59| 59.68935230623153|
|ATL-EWR|          20221026| 20221021|  68.6| 58.68260177673353|
|ATL-EWR|          20221027| 20221019| 72.59|  68.7014445522757|
|ATL-EWR|          20221027| 20221020|  68.6| 67.85245886419106|
|ATL-EWR|          20221027| 20221021|  68.6| 64.92072286053926|
|ATL-EWR|          20221029| 20221022|  68.6| 60.81067207918256|
|ATL-EWR|          20221030| 20221021|  68.6|  75.5736277042584|
|ATL-EWR|          20221030| 20221024| 107.6| 77.91242383628223|
|ATL-EWR|          20221031| 20221018|  68.6|60.362345969818605|
|ATL-EWR|          20221031| 20221025|  68.6| 67.69407317361086|
|ATL-EWR|          20221101| 20221024|  68.6| 62.46751204715671|
|ATL-EWR|          20221101| 20221025| 43.59| 59.65130929230471|
|ATL-EWR|          202211

In [20]:
pred_cv = calc_abs_pct_err(pred_cv)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-20:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [21]:
pred_cv.select("market", "out_departure_date", "shop_date", "fare", "prediction", "pred_err", "abs_pred_err", "abs_pct_err").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-21:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------+------------------+---------+------+------------------+-------------------+------------------+--------------------+
| market|out_departure_date|shop_date|  fare|        prediction|           pred_err|      abs_pred_err|         abs_pct_err|
+-------+------------------+---------+------+------------------+-------------------+------------------+--------------------+
|ATL-EWR|          20221025| 20221019| 72.59| 59.68935230623153| 12.900647693768477|12.900647693768477|  0.1777193510644507|
|ATL-EWR|          20221026| 20221021|  68.6| 58.68260177673353|  9.917398223266467| 9.917398223266467|  0.1445684872196278|
|ATL-EWR|          20221027| 20221019| 72.59|  68.7014445522757| 3.8885554477243005|3.8885554477243005| 0.05356874841884971|
|ATL-EWR|          20221027| 20221020|  68.6| 67.85245886419106| 0.7475411358089303|0.7475411358089303| 0.01089710110508645|
|ATL-EWR|          20221027| 20221021|  68.6| 64.92072286053926|  3.679277139460737| 3.679277139460737|0.053633777543159435|


In [22]:
pred_cv.select(F.mean("abs_pct_err").alias("mape")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-22:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------------------+
|               mape|
+-------------------+
|0.15039315031966055|
+-------------------+

In [23]:
pred_cv.groupBy("market").agg(F.mean("abs_pct_err").alias("mape")).orderBy("mape").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-23:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------+-------------------+
| market|               mape|
+-------+-------------------+
|LHR-EWR|0.04463653021047079|
|LHR-JFK|0.06025625193495919|
|EWR-LHR|0.08631058966526624|
|LHR-LAX|0.10302899553423048|
|JFK-LHR|0.11327558859594533|
|LAX-EWR| 0.1255679229352553|
|LAX-JFK| 0.1296015582648676|
|JFK-LAX|0.13072139929501464|
|EWR-CDG| 0.1324766548130451|
|ATL-EWR|0.15610776124039252|
|LGA-MIA|0.23158885639308674|
|LAX-SFO| 0.2586462839090909|
|SFO-LAX| 0.2648927709924579|
|OAK-LAS|0.42577761405333125|
+-------+-------------------+

In [24]:
cv_model.params

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[Param(parent='CrossValidatorModel_0afa70e8544f', name='estimator', doc='estimator to be cross-validated'), Param(parent='CrossValidatorModel_0afa70e8544f', name='estimatorParamMaps', doc='estimator param maps'), Param(parent='CrossValidatorModel_0afa70e8544f', name='evaluator', doc='evaluator used to select hyper-parameters that maximize the validator metric'), Param(parent='CrossValidatorModel_0afa70e8544f', name='seed', doc='random seed.')]

Exception in thread cell_monitor-24:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [28]:
cv_model.estimatorParamMaps

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Param(parent='CrossValidatorModel_0afa70e8544f', name='estimatorParamMaps', doc='estimator param maps')

Exception in thread cell_monitor-28:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [33]:
cv_model.extractParamMap()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{Param(parent='CrossValidatorModel_0afa70e8544f', name='seed', doc='random seed.'): 7809051150349531440, Param(parent='CrossValidatorModel_0afa70e8544f', name='estimator', doc='estimator to be cross-validated'): Pipeline_a2ad036ad7a2, Param(parent='CrossValidatorModel_0afa70e8544f', name='estimatorParamMaps', doc='estimator param maps'): [{Param(parent='LinearRegression_62af007e8b1d', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.01, Param(parent='LinearRegression_62af007e8b1d', name='regParam', doc='regularization parameter (>= 0).'): 0.01}, {Param(parent='LinearRegression_62af007e8b1d', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.01, Param(parent='LinearRegression_62af007e8b1d', name='regParam', doc='regularization parameter (>= 0).'): 0.1}, {Param

Exception in thread cell_monitor-33:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [41]:
x = cv_model.extractParamMap()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-41:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [29]:
cv_model.avgMetrics

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[135.84478252656768, 135.79289018511028, 135.63214047825554, 136.2831151009301, 139.92296898221207, 135.8459833213822, 135.70303015929642, 135.79741834072743, 136.31050253723407, 141.5253707283952, 135.70414446751784, 135.70864271321676, 135.84492537402036, 136.65829883729924, 151.8152590653045, 135.70415009565505, 135.80813764653433, 135.78519076347663, 137.45479672781923, 171.54772050397077, 135.70415274378956, 135.8077072884073, 135.71874294888184, 137.8387338504744, 175.81374590116377]

Exception in thread cell_monitor-29:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [36]:
# cv_results = list(zip(cv_model.estimatorParamMaps, cv_model.avgMetrics))

cv_results = list(zip(cv_model.getEstimatorParamMaps(), cv_model.avgMetrics))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-36:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [48]:
cv_results[0][0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{Param(parent='LinearRegression_62af007e8b1d', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.01, Param(parent='LinearRegression_62af007e8b1d', name='regParam', doc='regularization parameter (>= 0).'): 0.01}

Exception in thread cell_monitor-48:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [57]:
x = list(cv_results[0][0].keys())[0]
type(x)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pyspark.ml.param.Param'>

Exception in thread cell_monitor-57:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [59]:
x.name

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'elasticNetParam'

Exception in thread cell_monitor-59:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [54]:
cv_results[0][0].values()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

dict_values([0.01, 0.01])

Exception in thread cell_monitor-54:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [55]:
cv_results[0][0].params

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
'dict' object has no attribute 'params'
Traceback (most recent call last):
AttributeError: 'dict' object has no attribute 'params'

Exception in thread cell_monitor-55:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 

In [49]:
type(cv_results[0][0])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'dict'>

Exception in thread cell_monitor-49:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [60]:
cv_results.sort(key=lambda x: x[1])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-60:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [61]:
cv_results[0]

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

({Param(parent='LinearRegression_62af007e8b1d', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.01, Param(parent='LinearRegression_62af007e8b1d', name='regParam', doc='regularization parameter (>= 0).'): 1.0}, 135.63214047825554)

Exception in thread cell_monitor-61:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [62]:
best_result = cv_results[0]
best_metric = best_result[1]
best_metric

VBox()

Exception in thread cell_monitor-62:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

135.63214047825554

In [65]:
params = [x.name for x in list(best_result[0].keys())]
param_vals = best_result[0].values()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-65:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [67]:
best_params = dict(zip(params, param_vals))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-67:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [69]:
best_params, best_metric

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

({'elasticNetParam': 0.01, 'regParam': 1.0}, 135.63214047825554)

Exception in thread cell_monitor-69:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



# try a different model type

In [71]:
from pyspark.ml.regression import RandomForestRegressor

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-71:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [76]:
raw_feature_assembler = VectorAssembler(
    inputCols=all_features,
    outputCol='features'
)

regression_model = RandomForestRegressor(
    labelCol="fare",
    numTrees=50,
    maxDepth=10,
)

rf_pipeline = Pipeline(stages=[
    # scale_assembler,
    # norm_assembler,
    # ohe, 
    # scaler,
    # normer,
    raw_feature_assembler,
    regression_model
])

rf_model = rf_pipeline.fit(train_shuffled)
test_rf_tx = rf_model.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-76:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [77]:
test_rf_tx = calc_abs_pct_err(test_rf_tx)
test_rf_tx.select(F.mean("abs_pct_err").alias("mape")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-77:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------------------+
|               mape|
+-------------------+
|0.13308420519822364|
+-------------------+

A tiny bit better

In [78]:
test_rf_tx.groupBy("market").agg(F.mean("abs_pct_err").alias("mape")).orderBy("mape").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-78:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------+--------------------+
| market|                mape|
+-------+--------------------+
|LHR-EWR| 0.03601863988701263|
|LHR-JFK|0.057528979278835914|
|EWR-LHR|   0.064339470474955|
|JFK-LHR| 0.09352950501069193|
|LAX-EWR| 0.10154758151650022|
|EWR-CDG| 0.11288659273284089|
|LHR-LAX| 0.11337571493333565|
|JFK-LAX| 0.11884334524102143|
|LAX-JFK| 0.11982202249898405|
|ATL-EWR| 0.12641200907072284|
|LGA-MIA|  0.1952672733888646|
|LAX-SFO|  0.2028225565392824|
|SFO-LAX| 0.20561546207870865|
|OAK-LAS|  0.4952043019033468|
+-------+--------------------+

In [79]:
param_grid = (ParamGridBuilder()
              # regParam is lambda (weight / factor), elasticNet is alpha (ratio)
              # note alpha is the opposite of sklearn's defintion
              .addGrid(regression_model.numTrees, [50, 100, 200])
              .addGrid(regression_model.maxDepth, [3, 5, 7, 10])
              .build()
             )

cv = CrossValidator(estimator=rf_pipeline,
                    estimatorParamMaps=param_grid,
                    evaluator=RegressionEvaluator(labelCol="fare"),
                    numFolds=3,
                    parallelism=5
                   )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-79:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [80]:
cv_model = cv.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-80:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [81]:
pred_cv = cv_model.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-81:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [82]:
pred_cv = calc_abs_pct_err(pred_cv)
pred_cv.select(F.mean("abs_pct_err").alias("mape")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-82:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------------------+
|               mape|
+-------------------+
|0.13728407495010508|
+-------------------+

Worse!

In [83]:
pred_cv.groupBy("market").agg(F.mean("abs_pct_err").alias("mape")).orderBy("mape").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-83:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



+-------+-------------------+
| market|               mape|
+-------+-------------------+
|LHR-EWR|0.03879280927780875|
|LHR-JFK|0.05583507142016448|
|EWR-LHR|0.07129838774736268|
|JFK-LHR|0.09648400306355556|
|LHR-LAX|0.10177893301445474|
|LAX-EWR|0.10425436292823402|
|EWR-CDG|0.11884792408950245|
|LAX-JFK|0.12084846678811713|
|JFK-LAX|0.12106735555728934|
|ATL-EWR|0.13327181418026415|
|LAX-SFO|0.21271423216878468|
|LGA-MIA|0.21383080579332997|
|SFO-LAX| 0.2145632698261908|
|OAK-LAS| 0.5021130688304055|
+-------+-------------------+

In [85]:
cv_results = list(zip(
    cv_model.getEstimatorParamMaps(), 
    cv_model.avgMetrics
))

cv_results.sort(key=lambda x: x[1])
best_result = cv_results[0]
best_metric = best_result[1]
best_param_dict = best_result[0]
params = [x.name for x in list(best_param_dict.keys())]
param_vals = best_param_dict.values()
best_params = dict(zip(params, param_vals))
best_params, best_metric

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

({'numTrees': 100, 'maxDepth': 7}, 132.24015638150163)

Exception in thread cell_monitor-85:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



In [86]:
cv_model.avgMetrics

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[135.40395973335808, 132.93301068242337, 132.28642093206432, 133.4233221096628, 135.46124516821084, 133.01136754693533, 132.24015638150163, 134.06676966099332, 135.3185673654168, 132.93250354621387, 132.2436975298888, 133.59098721737925]

Exception in thread cell_monitor-86:
Traceback (most recent call last):
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/mnt/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in cell_monitor
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
  File "/mnt/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 154, in <listcomp>
    job_group_filtered_jobs = [job for job in jobs_data if job['jobGroup'] == str(statement_id)]
KeyError: 'jobGroup'



# feature selection

Alt: try PCA on all our similar crappy features

# split out by market